# PotassiumChannel

## 电生理特性

钾离子通道是神经元中最主要的超极化通道，决定了动作电位的恢复过程与静息膜电位的稳定性。其在神经元中的功能包括：

- 动作电位的复极：动作电位过程中，钠离子通道引起去极化，而钾通道开启导致钾外流，从而使膜电位回到静息水平。
- 控制兴奋性：某些钾通道具有延迟激活或钙依赖性质，能调节神经元的放电频率、适应性和持续兴奋性。
- 调控网络节律：有一些钾通道在节律放电、震荡中扮演重要角色。

钾通道功能广泛，其动力学建模在计算神经科学中具有重要地位。

## 建模实现

在 `braincell` 中，我们通过继承 `PotassiumChannel` 来实现钾通道的建模。该类继承自 `Channel`。

实现具体钾通道时，与实现钙通道相同，我们通常需要设定与钙通道相似的几个接口。

我们来看一个具体实例 `IKNI_Ya1989` ，便于更好的理解。

该模型最早由 Yamada 在 1989 年提出，是一类经典的缓慢非灭活钾离子通道，用于解释 AHP 期间的频率调节机制。

该模型由下列公式描述：

$$
\begin{aligned}
I_{M} &= \bar{g}_M \cdot p \cdot (V - E_K) \\\\
\frac{dp}{dt} &= \frac{p_{\infty}(V) - p}{\tau_p(V)} \\\\
p_{\infty}(V) &= \frac{1}{1 + \exp\left(-\frac{V - V_{\text{sh}} + 35}{10}\right)} \\\\
\tau_p(V) &= \frac{\tau_{\max}}{3.3 \exp\left(\frac{V - V_{\text{sh}} + 35}{20}\right) + \exp\left(-\frac{V - V_{\text{sh}} + 35}{20}\right)}
\end{aligned}
$$

其中：
- $V$ 是膜电位
- $E_K$ 是钾离子反转电位
- $\bar{g}_M$ 是最大电导
- $p$ 是门控变量
- $V_{\text{sh}}$ 是膜电位偏移
- $\tau_{\max}$ 是最大时间常数
- $p_{\infty}$ 是稳态激活函数
- $\tau_p$ 是激活时间常数

以下是代码实现：

```python
class IKNI_Ya1989(PotassiumChannel):
    def __init__(
        self,
        size,
        g_max=0.004 * (u.mS * u.cm ** -2),
        phi_p=1.,
        phi_q=1.,
        tau_max=4e3 * u.ms,
        V_sh=0. * u.mV,
        name=None,
    ):
        super().__init__(size=size, name=name)

        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.tau_max = brainstate.init.param(tau_max, self.varshape, allow_none=False)
        self.V_sh = brainstate.init.param(V_sh, self.varshape, allow_none=False)
        self.phi_p = brainstate.init.param(phi_p, self.varshape, allow_none=False)
        self.phi_q = brainstate.init.param(phi_q, self.varshape, allow_none=False)

    def init_state(self, V, Ca: IonInfo, batch_size: int = None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, K: IonInfo, batch_size=None):
        self.p.value = self.f_p_inf(V)
        if isinstance(batch_size, int):
            assert self.p.value.shape[0] == batch_size

    def compute_derivative(self, V, K: IonInfo):
        self.p.derivative = self.phi_p * (self.f_p_inf(V) - self.p.value) / self.f_p_tau(V)

    def current(self, V, K: IonInfo):
        return self.g_max * self.p.value * (K.E - V)

    def f_p_inf(self, V):
        V = (V - self.V_sh).to_decimal(u.mV)
        return 1. / (1. + u.math.exp(-(V + 35.) / 10.))

    def f_p_tau(self, V):
        V = (V - self.V_sh).to_decimal(u.mV)
        temp = V + 35.
        return self.tau_max / (3.3 * u.math.exp(temp / 20.) + u.math.exp(-temp / 20.))
```

从这个例子可以看出，`PotassiumChannel` 提供了统一的接口，而 `IKNI_Ya1989` 可以通过简单的数学表达式，就能轻松构建一个具备具体生物意义的离子通道模型。

当然，在实际应用中，我们还可以在子类下进一步构建子类。

比如，我们先构建 `IKK2_pq_ss` ,其继承自 `PotassiumChannel` 。而在后续工作中，我们可以构建继承自  `IKK2_pq_ss` 的 `KK2A_HM1992` ，并且在 `KK2A_HM1992` 中对原本的 `IKK2_pq_ss` 进行拓展。这样的流程方便大批量建模，提高工作效率。

如果你对这一部分感兴趣，可以参考我们的API文档。